In [1]:
import sys
import os
import glob
import time
import traceback
import datetime as dt

import gspread
from oauth2client.service_account import ServiceAccountCredentials

import numpy as np
import pandas as pd
from nd2reader import ND2Reader 
from scipy import ndimage as ndi
from scipy import stats
import pims
from PIL import Image

In [2]:
# Access google drive spreadsheet
scope = ['https://spreadsheets.google.com/feeds',
     'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('SenseOfTouchResearch-e5927f56c4d0.json', scope)
c = gspread.authorize(credentials)
metadataSpread = c.open_by_key('1LsTdPBOW79XSkk5DJv2ckiVTvpofAOOd_dJL4cgtxBQ')
metadataFrame = pd.DataFrame(metadataSpread.sheet1.get_all_records())
display(metadataFrame) # can propably do everything with this dataframe instead of moving to lists

,Bleach Date,Bleach Time,Cultivation Temperature (°C),Device ID,Email Address,Experiment ID,Life stage,Microscope,Neuron,Notes,Number of eggs visible,Objective,Purpose of Experiment,Timestamp,Worm Strain,Worm head orientation,Worm vulva orientation
0,4/25/2017,10:30:00 AM,20 Celsius,Y5P12-10,anekimke@stanford.edu,SSN_002_001,Day 1 adult,,AVM,"Worms too big, lots of autofluorescence",,,,4/27/2017 14:17:55,GN402 (MEC-4::YFP),,
1,5/2/2017,9:30:00 AM,20,Y5P12-5,anekimke@stanford.edu,SSN_003_001,YA,,ALM,"Worm escaped, so video shows nothing",,,Baseline measurement,5/4/2017 15:03:37,GN402,,
2,5/2/2017,9:30:00 AM,20,Y5P12-5,anekimke@stanford.edu,SSN_004_001,YA,,AVM,Worm strains might be mixed up. This looks lik...,,,Baseline measurement,5/4/2017 16:14:10,Tu2769,,
3,5/8/2017,10:00:00 AM,20,Y5P12-7,anekimke@stanford.edu,SSN_005_001,L4,,ALM,"No actuation this time, just trying to image w...",,,Baseline measurement,5/10/2017 12:00:42,GN402,,
4,5/14/2017,10:30:00 AM,20,Y5P12-2,anekimke@stanford.edu,SSN_006_001,D1A,,AVM,,,,Baseline measurement,5/17/2017 16:33:58,TU2769,,
5,5/14/2017,10:30:00 AM,20,Y5P12-2,anekimke@stanford.edu,SSN_006_002,D1A,,AVM,Switched Actuator to other side. Can't really ...,,,Baseline measurement,5/17/2017 16:37:59,TU2769,,
6,5/14/2017,10:30:00 AM,20,Y5P12-2,anekimke@stanford.edu,SSN_007_001,D1A,,AVM,Can't see neurite still,,,Baseline measurement,5/17/2017 16:43:47,TU2769,,
7,5/14/2017,10:30:00 AM,20,Y5P12-2,anekimke@stanford.edu,SSN_007_002,D1A,,AVM,in brightfield to see actuator in action,,,Baseline measurement,5/17/2017 16:55:40,TU2769,,
8,5/16/2017,9:30:00 AM,20,Y5P12-7,anekimke@stanford.edu,SSN_008_001,YA,,ALM,Actuator just behind ALM cell body,,,Baseline measurement,5/18/2017 11:49:08,TU2769,,
9,5/16/2017,9:30:00 AM,20,Y5P12-7,anekimke@stanford.edu,SSN_008_002,YA,,AVM,"Dim signal, but otherwise good. Actuator on op...",,,Baseline measurement,5/18/2017 11:54:01,TU2769,,


In [21]:
timestamps = metadataFrame['Timestamp']
dates_list = [dt.datetime.strptime(date, '%m/%d/%Y %H:%M:%S') for date in timestamps]

experiment_ids = metadataFrame['Experiment ID']
experiment_ids_worm_num = [experiment_ids[index][0:7] for index in range(len(experiment_ids))]

sdc_in_shriram_start = dt.datetime(2018, 9, 13, 14, 19, 9)
prime95b_camera_start = dt.datetime(2018, 11, 21, 13, 17, 36)

old_camera_sdc_trials = [experiment_ids[index] for index in range(len(experiment_ids)) if dates_list[index] > sdc_in_shriram_start and dates_list[index] < prime95b_camera_start]
old_camera_sdc_worms = [experiment_ids_worm_num[index] for index in range(len(experiment_ids)) if dates_list[index] > sdc_in_shriram_start and dates_list[index] < prime95b_camera_start]



new_camera_sdc_trials = [experiment_ids[index] for index in range(len(experiment_ids)) if dates_list[index] >= prime95b_camera_start] 
new_camera_sdc_worms = [experiment_ids_worm_num[index] for index in range(len(experiment_ids)) if dates_list[index] >= prime95b_camera_start] 

